<a href="https://colab.research.google.com/github/Estebanarci2/Marketing-analitica3/blob/main/D)DESPLIEGUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Sistema de recomendación filtro colaborativo**

In [5]:
!pip install scikit-surprise

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import sqlite3 as sql
import joblib
from ipywidgets import interact
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import MinMaxScaler
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from sklearn import neighbors
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
conn = sql.connect ("/content/drive/MyDrive/analitica3/marketing /db_movies_2")
cur=conn.cursor()

In [9]:
cur.execute("Select name from sqlite_master where type='table'")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('usuarios_fil',),
 ('movies_sel',),
 ('movies_fil',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('reco',)]

In [17]:
#Se realiza un filtro en cuanto a las calificaciones para que no tome
#calificaciones de cero en caso de que existan en otros estudios

pd.read_sql('select * from ratings_final where rating>0', conn)
ratings=pd.read_sql('select * from ratings_final where rating>0', conn)
reader = Reader(rating_scale=(0, 5)) ### la escala de la calificación

###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

In [18]:
#### función para probar varios modelos ##########
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.664289,0.872005,0.277871,1.732193
knns.KNNWithMeans,0.672616,0.882564,0.098125,1.078970
knns.KNNWithZScore,0.668649,0.882865,0.187235,1.210676
knns.KNNBasic,0.718074,0.941899,0.141209,1.200223


In [12]:
param_grid = {
    'sim_options': {
        'name': ['msd', 'cosine'],
        'min_support': [5],
        'user_based': [False, True]
    },
    'bsl_options': {
        'method': ['als', 'sgd'],  # Opciones para KNNBaseline
    }
}

In [19]:
### se afina si es basado en usuario o basado en ítem

gridsearchKNNBaseline = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=2)

gridsearchKNNBaseline.fit(data)

gridsearchKNNBaseline.best_params["rmse"]

gridsearchKNNBaseline.best_score["rmse"]

gs_model=gridsearchKNNBaseline.best_estimator['rmse'] ### mejor estimador de gridsearch
gs_model

In [20]:
################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)

predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y los movies que no han visto
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todos las peliculas que no ha visto un usuario
### la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape

predictions_df.head()

predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


,uid,iid,r_ui,est,details
0,1,318,3.704335,5.0,"{'actual_k': 40, 'was_impossible': False}"
514370,452,1234,3.704335,5.0,"{'actual_k': 40, 'was_impossible': False}"
514561,452,1175,3.704335,5.0,"{'actual_k': 11, 'was_impossible': False}"
514560,452,866,3.704335,5.0,"{'actual_k': 15, 'was_impossible': False}"
666059,584,175,3.704335,5.0,"{'actual_k': 11, 'was_impossible': False}"
...,...,...,...,...,...
503596,442,1499,3.704335,0.0,"{'actual_k': 2, 'was_impossible': False}"
277384,245,3438,3.704335,0.0,"{'actual_k': 1, 'was_impossible': False}"
3423,3,1499,3.704335,0.0,"{'actual_k': 6, 'was_impossible': False}"
3319,3,1792,3.704335,0.0,"{'actual_k': 3, 'was_impossible': False}"


In [21]:
##### funcion para recomendar las 10 películas con mejores predicciones y llevar base de datos para consultar
#resto de información
def recomendaciones(user_id,n_recomend=10):

    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")

    recomendados=pd.read_sql('''select a.*, b.title
                             from reco a left join movies_final b
                             on a.iid=b.movieId ''', conn)

    return(recomendados)

recomendacion = recomendaciones(user_id=23,n_recomend=10)
recomendacion

,index,iid,est,title
0,24944,1204,4.212892,Lawrence of Arabia (1962)
1,24265,1272,4.182459,Patton (1970)
2,24964,1104,4.169802,"Streetcar Named Desire, A (1951)"
3,24537,922,4.116119,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
4,24238,318,4.113763,"Shawshank Redemption, The (1994)"
5,25127,1235,4.086263,Harold and Maude (1971)
6,25170,2067,4.060042,Doctor Zhivago (1965)
7,24097,527,4.054968,Schindler's List (1993)
8,24709,1228,4.043482,Raging Bull (1980)
9,24291,898,4.037219,"Philadelphia Story, The (1940)"


In [22]:
#Se guarda la tabla en un archivo CSV descargable para
#facilitar la comprensión de las recomendaciones por parte del usuario

recomendacion.to_csv('recomendacion.csv', index=True)